# Customers and Products Analysis Using SQL

The goal of this project is to analyze data from a sales records database for scale model cars and extract information for decision-making.

During this project I searched for the answers for the following questions:
- Which products should we order more of or less of?
- How should we tailor marketing and communication strategies to customer behaviors?
- How much can we spend on acquiring new customers?

Before of all, I needed to analyze the structure of database and what informations are contained in the tables.

Database contains 8 tables in all and their brief description and their links are below:
- customers: customer data
	- PK: customerNumber
	- FK: salesRepEmployeeNumber --> employees (employeeNumber)
- employees: all employee information
	- PK: employeeNumber
	- FK: reportsTo --> employees (employeeNumber), officeCode --> offices (officeCode)
		
- offices: sales office information
	- PK: officeCode
		
- orders: customers' sales orders
	- PK: orderNumber
	- FK: customerNumber --> customers (customerNumber)
		
- orderDetails: sales order line for each sales order
	- PK: orderNumber, productCode
	- FK: orderNumber --> orders (orderNumber),  productCode --> products (productCode)
		
- payments: customers' payment records
	- PK: customerNumber, checkNumber
	- FK: customerNumber --> customers (customerNumber)
		
- products: a list of scale model cars
	- PK: productCode
	- FK: productLine --> productlines(productLine)
		
- productlines: a list of product line categories
	- PK: productLine

The following query showing number of rows and number of attributes of each table.

In [ ]:
SELECT "Customers" AS table_name,
				(SELECT COUNT(*)
					FROM pragma_table_info('customers')) AS number_of_attributes,
				COUNT(*) AS number_of_rows
  FROM customers

UNION ALL

SELECT "Products" AS table_name,
				(SELECT COUNT(*)
					FROM pragma_table_info('products')) AS number_of_attributes,
				COUNT(*) AS number_of_rows
  FROM products
  
UNION ALL

SELECT "ProductLines" AS table_name,
				(SELECT COUNT(*)
					FROM pragma_table_info('productlines')) AS number_of_attributes,
				COUNT(*) AS number_of_rows
  FROM productlines
  
UNION ALL

SELECT "Orders" AS table_name,
				(SELECT COUNT(*)
					FROM pragma_table_info('orders')) AS number_of_attributes,
				COUNT(*) AS number_of_rows
  FROM Orders
  
UNION ALL

SELECT "OrderDetails" AS table_name,
				(SELECT COUNT(*)
					FROM pragma_table_info('orderdetails')) AS number_of_attributes,
				COUNT(*) AS number_of_rows
  FROM orderdetails
  
UNION ALL

SELECT "Payments" AS table_name,
				(SELECT COUNT(*)
					FROM pragma_table_info('payments')) AS number_of_attributes,
				COUNT(*) AS number_of_rows
  FROM payments
  
UNION ALL

SELECT "Employees" AS table_name,
				(SELECT COUNT(*)
					FROM pragma_table_info('employees')) AS number_of_attributes,
				COUNT(*) AS number_of_rows
  FROM employees
  
UNION ALL

SELECT "Offices" AS table_name,
				(SELECT COUNT(*)
					FROM pragma_table_info('offices')) AS number_of_attributes,
				COUNT(*) AS number_of_rows
  FROM offices	

### Question 1

After analyzing structure of database I started answering the questions.
I have looked for products which are ordered more frequently because this help to optimize the supply and user experience by preventing that products from running out-of-stock. 

I have researched for two indicators in this question:
- `low stock` which represents that products that are almost or completely runned out-of-stock. This is can computed by sum of each product ordered divided by the quantity of product in stock
- `product performance` represents the sum of sales per products

Need to restocking products which have high product performance and close to running out-of-stock (prioritized products).

In [ ]:
-- Computing the low stock ratio per product (top 10)
SELECT od.productCode,
       (SELECT ROUND(SUM(od.quantityOrdered)*1.0 / pr.quantityInStock, 2)
          FROM products AS pr
         WHERE pr.productCode = od.productCode) AS low_stock
  FROM orderdetails AS od
 GROUP BY od.productCode
 ORDER BY low_stock DESC
 LIMIT 10

In [ ]:
-- Computing the performance indicator per product (top 10)
SELECT productCode,
       ROUND(SUM(quantityOrdered * priceEach), 2) AS product_performance
  FROM orderdetails
 GROUP BY productCode
 ORDER BY product_performance DESC
 LIMIT 10

I have combined the two queries above using with Common Table Expression and I have used them in two select then I stacked to get the prioritized products for restocking.

In [ ]:
WITH
top10_low_stock AS (
SELECT od.productCode,
       (SELECT ROUND(SUM(od.quantityOrdered)*1.0 / pr.quantityInStock, 2)
          FROM products AS pr
         WHERE pr.productCode = od.productCode) AS low_stock
  FROM orderdetails AS od
 GROUP BY od.productCode
 ORDER BY low_stock DESC
 LIMIT 10
),

top10_prod_performance AS (
SELECT productCode,
       ROUND(SUM(quantityOrdered * priceEach), 2) AS product_performance
  FROM orderdetails
 GROUP BY productCode
 ORDER BY product_performance DESC
 LIMIT 10
)

SELECT productName, productLine
  FROM products
 WHERE productCode IN (SELECT productCode
                         FROM top10_prod_performance) 
                         
UNION ALL
                         
SELECT productName, productLine
  FROM products
 WHERE productCode IN (SELECT productCode
                         FROM top10_low_stock) 
 LIMIT 10

### Question 2

In the next question I have investigated that how we can tailor for our customers the marketing strategy based on their behavior. For this I have explored the customers' information and categorized them into two category: 

- `VIP`s who bringing more profig for the store, and
- `Less-engaged` customers who bringing less profit

For this investigation I have used the following three tables:
- products
- orderdetails
- orders

In the first query inside this question I have gathered information about profits which are generated by customers.

In [ ]:
SELECT o.customerNumber AS customer_num,
       SUM(od.quantityOrdered * (od.priceEach - pr.buyPrice)) as profit
  FROM products AS pr
  JOIN orderdetails AS od
    ON pr.productCode = od.productCode
  JOIN orders AS o
    ON od.orderNumber = o.orderNumber
 GROUP BY o.customerNumber
 ORDER BY profit DESC

The output of the query here is below (just the first ten rows are showing)

After it I have listed the top 5 VIP customers with help of the previous query as CTE, based on profit.

In [ ]:
WITH 
-- CTE to profit per customer
profit_per_customer AS (
SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS profit
  FROM products p
  JOIN orderdetails od
    ON p.productCode = od.productCode
  JOIN orders o
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber
)

SELECT c.contactLastName AS lastname, c.contactFirstName AS firstname, c.city, c.country, ppc.profit
  FROM customers AS c
  JOIN profit_per_customer AS ppc
    ON c.customerNumber = ppc.customerNumber
 ORDER BY ppc.profit DESC
 LIMIT 5

And similar to this, I have listed the 'top 5' least-engaged customers by just changed the order in the main query.

In [ ]:
-- CTE as in the previous query

-- Main query with ascending order
SELECT c.contactLastName AS lastname, 
       c.contactFirstName AS first_name,
       c.city, c.country, ppc.profit
  FROM customers AS c
  JOIN profit_per_customer AS ppc
    ON c.customerNumber = ppc.customerNumber
 ORDER BY ppc.profit
 LIMIT 5

### Question 3

With this question I was looking for the answer how much money needed to spend to acquiring more new customers. For this I needed to compute the `Customer Lifetime Value` (LTV) which means the avergae amount of profit genereted by customer. After it it can be determine the marketing cost.

In [ ]:
WITH 
-- As the same CTE which used before
profit_per_customer AS (
SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS profit
  FROM products p
  JOIN orderdetails od
    ON p.productCode = od.productCode
  JOIN orders o
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber
)

-- Computing the average profit by customers
SELECT ROUND(AVG(ppc.profit), 2) AS ltv
  FROM profit_per_customer AS ppc

According to the query result `the profit` which the customers bringing `in average is 39039.59`.
Now that we know how much profit bringing by customers in average it can be computable how much money needed to cost acquiring new customers and need to spend on marketing.

### Conclusion

We can see the most performance products and that the Classic cars need to restocking because they are most frequently sell. Furthermore the knowledge about most-important and less-committed help us to drive the loyality and attract more customers. It can be imporve the marketing strategy. And finally it can be determine how much cost need to marketing to we can gain more new customers by knowing the average profit by them.